# Data cleaning passengers by terminal dataset - Team 86

The dataset downloaded from the platform datos.gov.co  (https://www.datos.gov.co/Transporte/Operaci-n-de-pasajeros-y-despacho-de-veh-culos-en-/eh75-8ah6) have completeness and standardization attributes, so the data cleaning is mainly focused on the transformation of the passenger variable and on the aggregation of other fields such as the name of the municipality and department.

In [1]:
# Import pandas library
import pandas as pd

In [2]:
# Load the dataset from datos.gov.co and transform it into a data frame

path = 'Operacion_de_pasajeros_terrestres_jun2021.csv'
df = pd.read_csv(path, parse_dates = ['FECHA_DESPACHO'], dtype = {"PASAJEROS" : "string"})

In [3]:
#If the 'PASSENGERS' characteristic is a string, apply the following cleaning on the data to convert the field to numeric
pasajeros = df["PASAJEROS"]
pasajeros = pasajeros.str.strip("")
pasajeros = pasajeros.str.replace(',','', regex = False)
df["PASAJEROS_CLEAN"] = pasajeros
df["PASAJEROS_CLEAN"] = pd.to_numeric(df["PASAJEROS_CLEAN"])

In [4]:
# remove the following outlier
df = df.drop(379052)

In [5]:
# Load the dataset of departments and municipalities of Colombia to extract the names
path = 'Departamentos_y_municipios_de_Colombia.csv'
df2 = pd.read_csv(path, dtype = {"CÓDIGO DANE DEL MUNICIPIO" : "string"})

In [6]:
# Transform the feature 'CÓDIGO DANE DEL MUNICIPIO' to numeric from the dataset about departments and municipalities
municipios = df2["CÓDIGO DANE DEL MUNICIPIO"]
municipios = municipios.str.replace(',','', regex = False)
df2["CÓDIGO DANE DEL MUNICIPIO_CLEAN"] = municipios
df2["CÓDIGO DANE DEL MUNICIPIO_CLEAN"] = pd.to_numeric(df2["CÓDIGO DANE DEL MUNICIPIO_CLEAN"])

In [7]:
# Create features and merge on dataframe 'df'
df['MUNICIPIO_ORIGEN'] = df['MUNICIPIO_ORIGEN_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['MUNICIPIO'])
df['DEPARTAMENTO_ORIGEN'] = df['MUNICIPIO_ORIGEN_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['DEPARTAMENTO'])
df['REGION_ORIGEN'] = df['MUNICIPIO_ORIGEN_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['REGION'])

df['MUNICIPIO_DESTINO'] = df['MUNICIPIO_DESTINO_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['MUNICIPIO'])
df['DEPARTAMENTO_DESTINO'] = df['MUNICIPIO_DESTINO_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['DEPARTAMENTO'])
df['REGION_DESTINO'] = df['MUNICIPIO_DESTINO_RUTA'].map(df2.set_index('CÓDIGO DANE DEL MUNICIPIO_CLEAN')['REGION'])

In [8]:
# Remove duplicate features
df=df.drop(columns = "PASAJEROS")

In [9]:
# validate the type of each variable
df.dtypes

TERMINAL                          object
CLASE_VEHICULO                    object
NIVEL_SERVICIO                    object
MUNICIPIO_ORIGEN_RUTA              int64
MUNICIPIO_DESTINO_RUTA             int64
FECHA_DESPACHO            datetime64[ns]
HORA_DESPACHO                      int64
TIPO_DESPACHO                     object
DESPACHOS                          int64
PASAJEROS_CLEAN                    int64
MUNICIPIO_ORIGEN                  object
DEPARTAMENTO_ORIGEN               object
REGION_ORIGEN                     object
MUNICIPIO_DESTINO                 object
DEPARTAMENTO_DESTINO              object
REGION_DESTINO                    object
dtype: object

In [10]:
# Validate the data
df.head(5)

,TERMINAL,CLASE_VEHICULO,NIVEL_SERVICIO,MUNICIPIO_ORIGEN_RUTA,MUNICIPIO_DESTINO_RUTA,FECHA_DESPACHO,HORA_DESPACHO,TIPO_DESPACHO,DESPACHOS,PASAJEROS_CLEAN,MUNICIPIO_ORIGEN,DEPARTAMENTO_ORIGEN,REGION_ORIGEN,MUNICIPIO_DESTINO,DEPARTAMENTO_DESTINO,REGION_DESTINO
0,T.T. DE BOGOTÁ NORTE,MICROBUS,LUJO,11001,15759,2021-03-01,8,TRANSITO,2,0,Bogotá D.C.,Bogotá D.C.,Región Centro Oriente,Sogamoso,Boyacá,Región Centro Oriente
1,T.T. DE BOGOTÁ SALITRE,BUS,LUJO,11001,76001,2021-03-12,11,ORIGEN,4,45,Bogotá D.C.,Bogotá D.C.,Región Centro Oriente,Cali,Valle del Cauca,Región Pacífico
2,T.T. DE CALI,BUSETA,BASICO,76001,52378,2021-01-09,6,ORIGEN,1,3,Cali,Valle del Cauca,Región Pacífico,La Cruz,Nariño,Región Pacífico
3,T.T. DE MELGAR,MICROBUS,BASICO,73449,25488,2021-02-28,12,ORIGEN,1,1,Melgar,Tolima,Región Centro Sur,Nilo,Cundinamarca,Región Centro Oriente
4,T.T. DE MANIZALES,MICROBUS,LUJO,17001,17174,2021-01-28,20,ORIGEN,1,13,Manizales,Caldas,Región Eje Cafetero - Antioquia,Chinchiná,Caldas,Región Eje Cafetero - Antioquia


In [11]:
# Export the clean data
df.to_csv('data_clean.csv', index=False)